# PREPARATION

## Import Required Library

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json
from pprint import pprint
import math

## Global Helper Function

In [2]:
def create_criteria_decision_form(total_criteria: int) -> None: 
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias['type'] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            print(criterias)
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [3]:
def create_matrix_form(rows: int, cols: int) -> None:
    col_labels = [""] + [f"C{i+1}" for i in range(cols)]
    row_labels = [f"A{i+1}" for i in range(rows)]
    
    inputs = [[widgets.FloatText(value=0, layout=widgets.Layout(width="50px")) 
               for c in range(cols)] for r in range(rows)]

    header_row = [widgets.Label(value=label, layout=widgets.Layout(width="50px")) for label in col_labels]
    
    input_rows = []
    for r in range(rows):
        row_widgets = [widgets.Label(value=row_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        
        decision_matrix.loc[:, :] = get_matrix()

        with output:
            print("Decision matrix:")
            print(decision_matrix)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Starter Matrix-Like Data Structure of Criterias and Alternatives

In [4]:
total_criteria = 5
total_alternative = 5

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

Define criterias

In [5]:
criterias = pd.DataFrame(None, index=criteria_labels, columns=['type'])
print(criterias)

   type
C1  NaN
C2  NaN
C3  NaN
C4  NaN
C5  NaN


Decide which criteria is benefit or cost

In [6]:
create_criteria_decision_form(total_criteria)

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Create matrix for decision

In [7]:
decision_matrix = pd.DataFrame(0, index=alternative_labels, columns=criteria_labels)

print(decision_matrix)

    C1  C2  C3  C4  C5
A1   0   0   0   0   0
A2   0   0   0   0   0
A3   0   0   0   0   0
A4   0   0   0   0   0
A5   0   0   0   0   0


In [8]:
create_matrix_form(rows=total_alternative, cols=total_criteria)

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Normalization

Formula:

**Benefit criteria:**
$$
r_{ij} = \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

**Cost criteria:**
$$
r_{ij} = 1 - \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

In [9]:
matrix_r = decision_matrix.copy(deep=True).astype(float)

for i in range(total_criteria):
    sum_col = (decision_matrix.iloc[:, i] ** 2).sum()

    for j, row in enumerate(decision_matrix.itertuples(index=False)):
        if criterias.iloc[i]['type'] == 'Benefit':
            matrix_r.iloc[j, i] = decision_matrix.iloc[j, i] / math.sqrt(sum_col) # benefit criteria normalization
        else:
            matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization

print(matrix_r)

          C1        C2        C3        C4        C5
A1  0.572397  0.440894  0.604161  0.330289  0.513870
A2  0.334840  0.480830  0.418265  0.440386  0.434813
A3  0.714931  0.600638  0.464739  0.550482  0.395285
A4  0.572397  0.640575  0.371792  0.495434  0.355756
A5  0.667420  0.640575  0.325318  0.385337  0.513870


# Calculate Normalization Matrix by Weights

In [ ]:
print('Calculation should be here')

# Get Concordance and Discordance Set

## Concordance

In [ ]:
print('Calculation should be here')

## Discordance

In [ ]:
print('Calculation should be here')

# Determine the Dominant Aggregate of The Matrix

In [ ]:
print('Calculation should be here')

# Eliminate Less Favourable Alternatives

In [ ]:
print('Calculation should be here')

# Rangking Alternative

In [ ]:
print('Calculation should be here')

---

# Conclusion